### Test Mnist_data in 0-9 numbers
#### RNN ( LSTM )
- Learning rate = 0.001
- Training_iteration = 100000
- Batch_size = 128
- Using BasicLSTMCell
- Using dynamic_rnn
- Cost function :softmax_cross_entropy_with_logits
- Gradient decent : AdamOptimizer
#### Finally, I get the 96% accuracy

In [1]:
#load data
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# number 1 to 10 data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/james/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/james/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#hyperparameter
lr = 0.001
training_iteration = 100000
batch_size = 128

input_size = 28
output_size = 10
time_step = 28
hidden_units = 128


In [3]:
#placeholder

xs = tf.placeholder(tf.float32,[None,time_step,input_size])
ys = tf.placeholder(tf.float32,[None,output_size])

In [4]:
#define weight and bias
Weight = {
    'in':tf.Variable(tf.random_normal([input_size,hidden_units])),
    'out':tf.Variable(tf.random_normal([hidden_units,output_size]))
}

Bias = {
    'in':tf.Variable(tf.constant(0.1,shape =[hidden_units])),
    'out':tf.Variable(tf.constant(0.1,shape =[output_size]))
}



In [5]:
#RNN
def RNN(x,weight,bias):
    x = tf.reshape(x,[-1,input_size])
    x_in = tf.matmul(x,weight['in'])+bias['in']
    x_in = tf.reshape(x_in,[-1,time_step,hidden_units])
    ########## start cell 
    lstm = tf.nn.rnn_cell.BasicLSTMCell(hidden_units)
    
    ###### init state
    _init_state = lstm.zero_state(batch_size,dtype=tf.float32)
    
    
    ########### 每一步output 都放在一个outputs的list中
    #####state 是迭代的，最后一个state就是finalstate
    
    outputs, final_state = tf.nn.dynamic_rnn(lstm,x_in, initial_state = _init_state,time_major=False)
    
    outputs = tf.unstack(tf.transpose(outputs,[1,0,2]))
    
    results = tf.matmul(outputs[-1],weight['out'])+bias['out']
    return results
    

In [6]:
#prediction
prediction = RNN(xs,Weight,Bias)

In [7]:
#cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=ys))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [8]:
#train
training_op = tf.train.AdamOptimizer(lr).minimize(cost)

In [9]:
def compute_accuracy(v_xs,v_ys):
    global prediction
    y_pred = sess.run(prediction,feed_dict={xs:v_xs})
    correct = tf.equal(tf.argmax(y_pred,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    result = sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys})
    return result

In [10]:
correct_pred = tf.equal(tf.argmax(prediction,1),(tf.argmax(ys,1)))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
#run
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    step = 0 
    while step*batch_size < training_iteration:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size,time_step,input_size])
        test_xs,test_ys = mnist.test.next_batch(batch_size)
        test_xs = test_xs.reshape([batch_size,time_step,input_size])
        sess.run(training_op, feed_dict={xs:batch_xs, ys:batch_ys})
        if step % 50 ==0:
             print(sess.run(accuracy, feed_dict={xs:test_xs,ys:test_ys}))
        step+=1

0.1328125
0.84375
0.875
0.8671875
0.90625
0.90625
0.9453125
0.9296875
0.9375
0.96875
0.9140625
0.9453125
0.9453125
0.9453125
0.984375
0.9609375
